In [ ]:
from analysisdatalink.datalink_ext import AnalysisDataLinkExt as AnalysisDataLink
from neuroglancer_annotation_ui.statebuilder import StateBuilder, ChainedStateBuilder
from annotationframeworkclient.infoservice import InfoServiceClient
from IPython.core.display import HTML
import pandas as pd
import numpy as np

In [ ]:
data_version = 160
config_name = 'pinky100'

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=os.path.expanduser('~/.config/{config_name}/.env'.format(config_name=config_name)))

dataset_name = os.getenv('DATASET_NAME')
sql_database_uri_base = os.getenv('MATERIALIZATION_DATABASE_URI_BASE')
synapse_table = os.getenv('AUTOMATED_SYNAPSE_TABLE')
soma_table = os.getenv('SOMA_TABLE')

In [ ]:
dl = AnalysisDataLink(dataset_name=dataset_name,
                      sqlalchemy_database_uri=sql_database_uri_base,
                      materialization_version=data_version,
                      verbose=False)

## State Builder and Materialized Data Queries

### Soma Points
The most basic version. Generate a neuroglancer state from a dataframe with a bunch of points.

In [ ]:
info = InfoServiceClient(dataset_name=dataset_name)
img_dict = {'img': info.image_source(format_for='neuroglancer')}
seg_dict = {'seg': info.pychunkedgraph_viewer_source(format_for='neuroglancer')}

annotation_map = {'somata': {'points':['pt_position']}}
point_builder = StateBuilder(image_sources=img_dict,
                             seg_sources=seg_dict,
                             annotation_layers=annotation_map)

HTML(point_builder.render_state(dl.query_cell_ids(soma_table), return_as='html'))

### Soma Points by Cell Type
In order to make this more responsive to data, we pivot the dataframe to make cell-type specific layers with different colors. If we wanted to not anticipate the results, we could always generate the annotation map from the dataframe itself.

In [ ]:
info = InfoServiceClient(dataset_name=dataset_name)
img_dict = {'img': info.image_source(format_for='neuroglancer')}
seg_dict = {'seg': info.pychunkedgraph_viewer_source(format_for='neuroglancer')}

annotation_map = {'e somata': {'points':['e'], 'color':'#ffffff'},
                  'i somata': {'points':['i'], 'color':'#5dc7d0'},
                  'g somata': {'points':['g'], 'color':'#011c7f'}}

cell_type_builder = StateBuilder(image_sources=img_dict,
                                 seg_sources=seg_dict,
                                 annotation_layers=annotation_map)

df = dl.query_cell_ids(soma_table)
dfp = df.pivot(values='pt_position', columns='cell_type')
HTML(cell_type_builder.render_state(dfp, return_as='html'))

### Soma Points by Cell Type v2
This is the same thing as above, but we collect points from multiple columns into the same layer.

In [ ]:
info = InfoServiceClient(dataset_name=dataset_name)
img_dict = {'img': info.image_source(format_for='neuroglancer')}
seg_dict = {'seg': info.pychunkedgraph_viewer_source(format_for='neuroglancer')}

annotation_map = {'neurons': {'points':['e', 'i'], 'color':'#ffffff'},
                  'glia': {'points':['g'], 'color':'#5dc7d0'}}
cell_type_builder = StateBuilder(image_sources=img_dict,
                                 seg_sources=seg_dict,
                                 annotation_layers=annotation_map)

df = dl.query_cell_ids(soma_table)
dfp = df.pivot(values='pt_position', columns='cell_type')
HTML(cell_type_builder.render_state(dfp, return_as='html'))

## Synapses
Build line annotations from a synapse query and highlight the specific neuron

In [ ]:
id_cell = df[df['cell_type']=='i'].iloc[10]['pt_root_id']

info = InfoServiceClient(dataset_name=dataset_name)
img_dict = {'img': info.image_source(format_for='neuroglancer')}
seg_dict = {'seg': info.pychunkedgraph_viewer_source(format_for='neuroglancer')}

annotation_map = {'synapses': {'lines':[['ctr_pt_position', 'post_pt_position']], 'color':'#FF1200'}}
selection_map = {'seg': ['pre_pt_root_id']}
fixed_selection = {'seg': [id_cell]}
presyn_builder = StateBuilder(image_sources=img_dict,
                                 seg_sources=seg_dict,
                                 selected_ids=selection_map,
                                 annotation_layers=annotation_map,
                                 )

pre_df = dl.query_synapses(synapse_table, pre_ids=[id_cell])

HTML(presyn_builder.render_state( pre_df, return_as='html'))

#### Chaining multiple dataframes and builders
If we want to put several builders together in a row with different dataframes, we can just pass the json state of one into the base state of the next. The ChainedStateBuilder is a convenience class that does that work for you automatically, while applying the standard render state options on the return value. Here, we use it to render a dataframe of presynaptic synapses and a dataframe of postsynaptic synapses.

In [ ]:
id_cell = df[df['cell_type']=='i'].iloc[16]['pt_root_id']

info = InfoServiceClient(dataset_name=dataset_name)
img_dict = {'img': info.image_source(format_for='neuroglancer')}
seg_dict = {'seg': info.pychunkedgraph_viewer_source(format_for='neuroglancer')}

pre_annotation_map = {'pre': {'lines':[['ctr_pt_position', 'post_pt_position']], 'color':'#FF1200'}}
fixed_selection = {'seg': [id_cell]}
presyn_builder = StateBuilder(image_sources=img_dict,
                              seg_sources=seg_dict,
                              fixed_selection=fixed_selection,
                              annotation_layers=pre_annotation_map,
                              )

post_annotation_map = {'post': {'lines':[['pre_pt_position', 'ctr_pt_position']], 'color':'#03d7ff'}}
postsyn_builder = StateBuilder(annotation_layers=post_annotation_map)
chained_builder = ChainedStateBuilder((presyn_builder, postsyn_builder))

pre_df = dl.query_synapses(synapse_table, pre_ids=[id_cell])
post_df = dl.query_synapses(synapse_table, post_ids=[id_cell])
HTML(chained_builder.render_state((pre_df, post_df), return_as='html'))